In [58]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import time
import math
import copy
from tkinter import Canvas, Frame, Scrollbar

class Connect4Game:
    def __init__(self, rows=6, cols=7):
        self.rows = rows
        self.cols = cols
        self.board = self.create_board()
        self.game_over = False
        self.turn = 0  # 0 for human, 1 for AI
        self.winner = None
        
    def create_board(self):
        return np.zeros((self.rows, self.cols), dtype=int)
    
    def drop_piece(self, col, piece):
        for row in range(self.rows-1, -1, -1):
            if self.board[row][col] == 0:
                self.board[row][col] = piece
                return row
        return -1
    
    def is_valid_location(self, col):
        return self.board[0][col] == 0
    
    def get_next_open_row(self, col):
        for row in range(self.rows-1, -1, -1):
            if self.board[row][col] == 0:
                return row
        return -1
    
    def winning_move(self, piece):
        # Horizontal
        for r in range(self.rows):
            for c in range(self.cols - 3):
                if all(self.board[r][c+i] == piece for i in range(4)):
                    return True
        # Vertical
        for r in range(self.rows - 3):
            for c in range(self.cols):
                if all(self.board[r+i][c] == piece for i in range(4)):
                    return True
        # Positive diagonal
        for r in range(self.rows - 3):
            for c in range(self.cols - 3):
                if all(self.board[r+i][c+i] == piece for i in range(4)):
                    return True
        # Negative diagonal
        for r in range(3, self.rows):
            for c in range(self.cols - 3):
                if all(self.board[r-i][c+i] == piece for i in range(4)):
                    return True
        return False
    
    def is_board_full(self):
        return all(self.board[0][c] != 0 for c in range(self.cols))
    
    def get_score(self, piece):
        score = 0
        opponent = 1 if piece == 2 else 2
        
        # Winning moves have highest priority
        if self.winning_move(piece):
            return 100000
        if self.winning_move(opponent):
            return -100000
            
        # Center preference
        center_col = self.cols // 2
        center_count = 0
        for r in range(self.rows):
            if self.board[r][center_col] == piece:
                center_count += 1
        score += center_count * 3
        
        # Evaluate all possible windows
        for r in range(self.rows):
            for c in range(self.cols):
                # Horizontal windows
                if c <= self.cols - 4:
                    window = [self.board[r][c+i] for i in range(4)]
                    score += self.evaluate_window(window, piece)
                # Vertical windows
                if r <= self.rows - 4:
                    window = [self.board[r+i][c] for i in range(4)]
                    score += self.evaluate_window(window, piece)
                # Positive diagonal
                if r <= self.rows - 4 and c <= self.cols - 4:
                    window = [self.board[r+i][c+i] for i in range(4)]
                    score += self.evaluate_window(window, piece)
                # Negative diagonal
                if r >= 3 and c <= self.cols - 4:
                    window = [self.board[r-i][c+i] for i in range(4)]
                    score += self.evaluate_window(window, piece)
        return score
    
    def evaluate_window(self, window, piece):
        score = 0
        opponent = 1 if piece == 2 else 2
        
        piece_count = window.count(piece)
        opponent_count = window.count(opponent)
        empty_count = window.count(0)
        
        if piece_count == 3 and empty_count == 1:
            score += 100
        elif piece_count == 2 and empty_count == 2:
            score += 5
        elif piece_count == 1 and empty_count == 3:
            score += 1
            
        if opponent_count == 3 and empty_count == 1:
            score -= 80
        elif opponent_count == 2 and empty_count == 2:
            score -= 4
            
        return score

class Node:
    def __init__(self, board, move=None, parent=None, value=0, alpha=-math.inf, beta=math.inf):
        self.board = copy.deepcopy(board)
        self.move = move
        self.parent = parent
        self.value = value
        self.alpha = alpha
        self.beta = beta
        self.children = []
        self.depth = 0 if parent is None else parent.depth + 1
        self.pruned = False
        self.x = 0  # إحداثيات للرسم
        self.y = 0  # إحداثيات للرسم
    
    def add_child(self, child):
        self.children.append(child)

class Connect4AI:
    def __init__(self, game, depth=4, use_alpha_beta=True):
        self.game = game
        self.depth = depth
        self.use_alpha_beta = use_alpha_beta
        self.nodes_expanded = 0
        self.pruned_nodes = 0
        self.execution_time = 0
        self.root_node = None
        self.last_move_tree = None
        self.total_nodes = 0
        
    def minimax(self, board, depth, alpha, beta, maximizing_player, node):
        self.nodes_expanded += 1
        self.total_nodes += 1
        
        # Terminal conditions
        if depth == 0 or board.winning_move(1) or board.winning_move(2) or board.is_board_full():
            value = board.get_score(2)  # AI is player 2
            node.value = value
            node.alpha = alpha
            node.beta = beta
            return value
            
        valid_locations = [c for c in range(board.cols) if board.is_valid_location(c)]
        
        if maximizing_player:
            value = -math.inf
            for col in valid_locations:
                # Create child node
                temp_board = copy.deepcopy(board)
                temp_board.drop_piece(col, 2)  # AI is player 2
                child_node = Node(temp_board, col, node, alpha=alpha, beta=beta)
                node.add_child(child_node)
                
                # Recursive call
                new_value = self.minimax(temp_board, depth-1, alpha, beta, False, child_node)
                value = max(value, new_value)
                child_node.value = new_value
                
                # Alpha-Beta Pruning
                if self.use_alpha_beta:
                    alpha = max(alpha, value)
                    child_node.alpha = alpha
                    child_node.beta = beta
                    
                    if alpha >= beta:
                        self.pruned_nodes += len(valid_locations) - valid_locations.index(col) - 1
                        child_node.pruned = True
                        break
            return value
        else:
            value = math.inf
            for col in valid_locations:
                # Create child node
                temp_board = copy.deepcopy(board)
                temp_board.drop_piece(col, 1)  # Human is player 1
                child_node = Node(temp_board, col, node, alpha=alpha, beta=beta)
                node.add_child(child_node)
                
                # Recursive call
                new_value = self.minimax(temp_board, depth-1, alpha, beta, True, child_node)
                value = min(value, new_value)
                child_node.value = new_value
                
                # Alpha-Beta Pruning
                if self.use_alpha_beta:
                    beta = min(beta, value)
                    child_node.alpha = alpha
                    child_node.beta = beta
                    
                    if value <= alpha:
                        self.pruned_nodes += len(valid_locations) - valid_locations.index(col) - 1
                        child_node.pruned = True
                        break
            return value
    
    def get_best_move(self):
        start_time = time.time()
        self.nodes_expanded = 0
        self.pruned_nodes = 0
        self.total_nodes = 0
        
        valid_locations = [c for c in range(self.game.cols) if self.game.is_valid_location(c)]
        if not valid_locations:
            return -1
            
        best_value = -math.inf
        best_col = valid_locations[0]
        self.root_node = Node(self.game, alpha=-math.inf, beta=math.inf)
        
        for col in valid_locations:
            temp_board = copy.deepcopy(self.game)
            temp_board.drop_piece(col, 2)
            child_node = Node(temp_board, col, self.root_node, alpha=-math.inf, beta=math.inf)
            self.root_node.add_child(child_node)
            
            value = self.minimax(temp_board, self.depth-1, -math.inf, math.inf, False, child_node)
            child_node.value = value
            
            if value > best_value:
                best_value = value
                best_col = col
                
            # Update alpha for root
            if self.use_alpha_beta:
                self.root_node.alpha = max(self.root_node.alpha, best_value)
        
        self.execution_time = time.time() - start_time
        self.last_move_tree = self.root_node
        return best_col

class TreeVisualizer:
    def __init__(self, tree_root, depth=4, algorithm="Minimax"):
        self.tree_root = tree_root
        self.depth = depth
        self.algorithm = algorithm
        
        # إنشاء نافذة كبيرة للشجرة
        self.tree_window = tk.Toplevel()
        self.tree_window.title(f"🌳 Decision Tree - Depth: {depth}, Algorithm: {algorithm}")
        self.tree_window.geometry("1400x900")
        
        # إطار رئيسي مع scrollbars
        main_frame = Frame(self.tree_window)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # إضافة scrollbars
        h_scrollbar = Scrollbar(main_frame, orient=tk.HORIZONTAL)
        h_scrollbar.pack(side=tk.BOTTOM, fill=tk.X)
        
        v_scrollbar = Scrollbar(main_frame)
        v_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        # Canvas كبير للرسم مع scrollbars
        self.canvas = Canvas(main_frame, 
                            bg="#1a1a2e",
                            scrollregion=(0, 0, 3000, 2000),  # منطقة كبيرة للرسم
                            xscrollcommand=h_scrollbar.set,
                            yscrollcommand=v_scrollbar.set)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        h_scrollbar.config(command=self.canvas.xview)
        v_scrollbar.config(command=self.canvas.yview)
        
        # معلومات الشجرة
        info_frame = Frame(self.tree_window, bg="#162447", height=60)
        info_frame.pack(fill=tk.X, padx=10, pady=5)
        info_frame.pack_propagate(False)
        
        tk.Label(info_frame, 
                text=f"🌳 DECISION TREE VISUALIZATION | Depth: {depth} | Algorithm: {algorithm} | Total Nodes: {self.count_nodes(tree_root)}",
                font=("Arial", 12, "bold"),
                fg="white",
                bg="#162447").pack(pady=10)
        
        # لوحة الألوان التوضيحية
        color_frame = Frame(self.tree_window, bg="#1f4068")
        color_frame.pack(fill=tk.X, padx=10, pady=2)
        
        colors_info = [
            ("🔴 MAX Node", "#ff6b6b"),
            ("🔵 MIN Node", "#4ecdc4"),
            ("🟣 Root Node", "#9d65c9"),
            ("🟢 Selected Move", "#2ecc71"),
            ("⚫ Pruned Node", "#95a5a6"),
        ]
        
        for text, color in colors_info:
            color_label = Frame(color_frame, bg="#1f4068")
            color_label.pack(side=tk.LEFT, padx=10)
            tk.Canvas(color_label, width=20, height=20, bg=color, highlightthickness=0, 
                     relief="ridge", bd=2).pack(side=tk.LEFT, padx=2)
            tk.Label(color_label, text=text, font=("Arial", 9), fg="white", bg="#1f4068").pack(side=tk.LEFT)
        
        # رسم الشجرة
        self.draw_tree()
        
        # إضافة zoom controls
        control_frame = Frame(self.tree_window, bg="#162447")
        control_frame.pack(fill=tk.X, padx=10, pady=5)
        
        tk.Button(control_frame, text="🔍 Zoom In", command=self.zoom_in, 
                 font=("Arial", 10), bg="#3498db", fg="white", 
                 activebackground="#2980b9", relief="raised", padx=10).pack(side=tk.LEFT, padx=5)
        tk.Button(control_frame, text="🔍 Zoom Out", command=self.zoom_out,
                 font=("Arial", 10), bg="#e74c3c", fg="white",
                 activebackground="#c0392b", relief="raised", padx=10).pack(side=tk.LEFT, padx=5)
        tk.Button(control_frame, text="🔄 Reset View", command=self.reset_view,
                 font=("Arial", 10), bg="#2ecc71", fg="white",
                 activebackground="#27ae60", relief="raised", padx=10).pack(side=tk.LEFT, padx=5)
        tk.Button(control_frame, text="💾 Save Image", command=self.save_image,
                 font=("Arial", 10), bg="#9b59b6", fg="white",
                 activebackground="#8e44ad", relief="raised", padx=10).pack(side=tk.LEFT, padx=5)
        
        self.zoom_level = 1.0
        
        # توجيه تلقائي إلى منتصف الشجرة
        self.tree_window.after(100, self.center_tree)
    
    def count_nodes(self, node):
        """حساب عدد العقد في الشجرة"""
        if not node:
            return 0
        count = 1
        for child in node.children:
            count += self.count_nodes(child)
        return count
    
    def calculate_positions(self, node, x=1500, y=100, level=0):
        """حساب مواقع العقد للرسم مع جعل الشجرة أكبر"""
        if not node:
            return
            
        node.x = x
        node.y = y
        
        if node.children:
            # حساب المسافة بين الأطفال - جعلها أكبر
            children_count = len(node.children)
            
            # زيادة المسافة الأفقية مع العمق
            base_x_spacing = 400 - (level * 50)
            x_spacing = max(base_x_spacing, 150)
            
            # زيادة المسافة العمودية
            y_spacing = 180
            
            total_width = (children_count - 1) * x_spacing
            start_x = x - total_width / 2
            
            for i, child in enumerate(node.children):
                child_x = start_x + i * x_spacing
                child_y = y + y_spacing
                self.calculate_positions(child, child_x, child_y, level + 1)
    
    def draw_tree(self):
        """رسم الشجرة على Canvas"""
        self.canvas.delete("all")
        
        if not self.tree_root:
            self.canvas.create_text(1500, 1000, text="No tree data available", 
                                   fill="white", font=("Arial", 24, "bold"))
            return
        
        # حساب المواقع
        self.calculate_positions(self.tree_root)
        
        # رسم الخطوط أولاً (خلف العقد)
        self.draw_connections(self.tree_root)
        
        # رسم العقد
        self.draw_node(self.tree_root)
        
        # رسم الأطفال
        for child in self.tree_root.children:
            self.draw_subtree(child)
    
    def draw_connections(self, node):
        """رسم خطوط الوصل بين العقد"""
        for child in node.children:
            # رسم خط من العقدة الأم إلى الطفل
            line_color = "#7f8c8d" if child.pruned else "#a5b1c2"
            line_width = 2 if child.pruned else 3
            
            # جعل الخطوط أكثر سماكة
            self.canvas.create_line(node.x, node.y + 40, 
                                   child.x, child.y - 40,
                                   fill=line_color, width=line_width, tags="connection",
                                   arrow="last", arrowshape=(16, 20, 8))
            
            # رسم الخطوط للأحفاد
            self.draw_connections(child)
    
    def draw_subtree(self, node):
        """رسم شجرة فرعية"""
        if not node:
            return
        
        self.draw_node(node)
        
        for child in node.children:
            self.draw_subtree(child)
    
    def draw_node(self, node):
        """رسم عقدة واحدة بشكل كبير"""
        # تحديد لون العقدة بناءً على نوعها
        if node.depth == 0:  # الجذر
            color = "#9d65c9"  # أرجواني
            border_color = "#8a56ac"
            text_color = "white"
            shadow_color = "#6d3d8c"
        elif node.depth % 2 == 0:  # MAX
            color = "#ff6b6b"  # أحمر
            border_color = "#ff5252"
            text_color = "white"
            shadow_color = "#d63031"
        else:  # MIN
            color = "#4ecdc4"  # أزرق فاتح
            border_color = "#45b7aa"
            text_color = "white"
            shadow_color = "#00b894"
            
        # إذا كانت العقدة مقصوصة (pruned)
        if node.pruned:
            color = "#95a5a6"  # رمادي
            border_color = "#7f8c8d"
            text_color = "white"
            shadow_color = "#636e72"
        
        # إذا كانت هذه الحركة المختارة
        is_selected = False
        if node.depth == 1 and node.parent == self.tree_root:
            # العثور على أفضل طفل (ذو أعلى قيمة للجذر MAX)
            if self.tree_root.children:
                best_child = max(self.tree_root.children, key=lambda x: x.value)
                if node == best_child:
                    color = "#2ecc71"  # أخضر
                    border_color = "#27ae60"
                    text_color = "white"
                    shadow_color = "#219653"
                    is_selected = True
        
        # رسم تأثير الظل
        shadow_offset = 4
        self.canvas.create_oval(node.x - 50 + shadow_offset, node.y - 50 + shadow_offset,
                               node.x + 50 + shadow_offset, node.y + 50 + shadow_offset,
                               fill=shadow_color, outline=shadow_color, width=0, tags="shadow")
        
        # رسم الدائرة (العقدة) - جعلها أكبر
        radius = 50  # زيادة حجم العقدة
        self.canvas.create_oval(node.x - radius, node.y - radius,
                               node.x + radius, node.y + radius,
                               fill=color, outline=border_color, width=4, tags="node")
        
        # نص الحركة - جعله أكبر
        move_text = f"Col {node.move+1}" if node.move is not None else "🌳 Root"
        self.canvas.create_text(node.x, node.y - 80, 
                               text=move_text, 
                               fill="white", font=("Arial", 14, "bold"), tags="text")
        
        # نص القيمة - جعله أكبر
        value_text = f"Score: {node.value:.0f}"
        self.canvas.create_text(node.x, node.y, 
                               text=value_text, 
                               fill=text_color, 
                               font=("Arial", 16, "bold"), tags="text")
        
        # معلومات العمق
        depth_text = f"Depth: {node.depth}"
        self.canvas.create_text(node.x, node.y + 65, 
                               text=depth_text, 
                               fill="#dcdde1", font=("Arial", 10), tags="text")
        
        # معلومات Alpha-Beta إذا كانت متاحة
        if hasattr(node, 'alpha') and hasattr(node, 'beta'):
            ab_text = f"α={node.alpha:.0f} | β={node.beta:.0f}"
            self.canvas.create_text(node.x, node.y + 85, 
                                   text=ab_text, 
                                   fill="#74b9ff", font=("Arial", 9), tags="text")
    
    def zoom_in(self):
        """تكبير الرسم"""
        self.zoom_level *= 1.3
        self.canvas.scale("all", 1500, 1000, 1.3, 1.3)
    
    def zoom_out(self):
        """تصغير الرسم"""
        self.zoom_level /= 1.3
        self.canvas.scale("all", 1500, 1000, 1/1.3, 1/1.3)
    
    def reset_view(self):
        """إعادة تعيين العرض"""
        self.canvas.delete("all")
        self.zoom_level = 1.0
        self.draw_tree()
        self.center_tree()
    
    def center_tree(self):
        """توسيط الشجرة في العرض"""
        self.canvas.xview_moveto(0.3)
        self.canvas.yview_moveto(0.1)
    
    def save_image(self):
        """حفظ صورة الشجرة"""
        try:
            # لاحظ: في بيئة حقيقية، قد تحتاج إلى استخدام PIL لتصدير الصورة
            messagebox.showinfo("Save Image", 
                              "Feature would save tree as PNG image.\nIn this demo, please take a screenshot.")
        except Exception as e:
            messagebox.showerror("Error", f"Could not save image: {str(e)}")

class Connect4GUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Connect 4 AI - Large Tree Display")
        self.root.geometry("800x900")
        
        # استخدام ألوان أكثر جمالاً
        self.root.configure(bg="#2c3e50")
        
        self.game = None
        self.ai = None
        self.depth = 4
        self.algorithm = "minimax_ab"
        
        # إحصاءات اللعبة
        self.human_wins = 0
        self.ai_wins = 0
        self.draws = 0
        self.total_moves = 0
        
        self.setup_menu()
    
    def setup_menu(self):
        self.clear_window()
        
        # إطار رئيسي للقائمة
        menu_frame = tk.Frame(self.root, bg="#34495e", bd=2, relief="ridge", padx=20, pady=20)
        menu_frame.pack(pady=40, padx=40, fill="both", expand=True)
        
        # العنوان الرئيسي
        title_label = tk.Label(menu_frame, 
                              text="🎮 Connect 4 AI", 
                              font=("Arial", 28, "bold"),
                              bg="#34495e",
                              fg="white")
        title_label.pack(pady=(10, 20))
        
        subtitle_label = tk.Label(menu_frame,
                                text="Large Tree Visualization Game",
                                font=("Arial", 14),
                                bg="#34495e",
                                fg="#bdc3c7")
        subtitle_label.pack(pady=(0, 30))
        
        # تعليمات اللعبة
        instructions_frame = tk.Frame(menu_frame, bg="#2c3e50", padx=15, pady=15, relief="sunken", bd=2)
        instructions_frame.pack(pady=(0, 20), fill="x")
        
        instructions = tk.Label(instructions_frame, 
                              text="🎯 Game Rules:\n• Game continues until board is completely filled\n• Click columns to drop your pieces\n• Click 'Show Large Tree' to see BIG graphical decision tree",
                              font=("Arial", 11),
                              bg="#2c3e50",
                              fg="white",
                              justify=tk.LEFT)
        instructions.pack()
        
        # عرض إحصائيات الفوز
        stats_frame = tk.Frame(menu_frame, bg="#34495e")
        stats_frame.pack(pady=15)
        
        stats_label = tk.Label(stats_frame, 
                              text="📊 Game Statistics", 
                              font=("Arial", 14, "bold"),
                              bg="#34495e",
                              fg="white")
        stats_label.pack()
        
        stats_text = tk.Label(stats_frame, 
                             text=f"👤 Human: {self.human_wins} wins  |  🤖 AI: {self.ai_wins} wins  |  🤝 Draws: {self.draws}", 
                             font=("Arial", 12, "bold"),
                             bg="#34495e",
                             fg="#f1c40f")
        stats_text.pack(pady=5)
        
        # إطار إعدادات العمق
        settings_frame = tk.Frame(menu_frame, bg="#34495e")
        settings_frame.pack(pady=20)
        
        # إعدادات العمق
        depth_frame = tk.LabelFrame(settings_frame, 
                                   text="🔧 Search Depth Settings", 
                                   font=("Arial", 12, "bold"),
                                   bg="#34495e",
                                   fg="white",
                                   padx=15,
                                   pady=10)
        depth_frame.pack(side=tk.LEFT, padx=10)
        
        self.depth_var = tk.IntVar(value=4)
        for depth in [2, 3, 4, 5, 6]:
            radio = tk.Radiobutton(depth_frame, 
                                  text=f"Depth {depth}", 
                                  variable=self.depth_var, 
                                  value=depth,
                                  font=("Arial", 11),
                                  bg="#34495e",
                                  fg="white",
                                  selectcolor="#2c3e50",
                                  activebackground="#34495e",
                                  activeforeground="#3498db")
            radio.pack(anchor="w", pady=2)
        
        # إعدادات الخوارزمية
        algo_frame = tk.LabelFrame(settings_frame, 
                                  text="⚙️ Algorithm Settings", 
                                  font=("Arial", 12, "bold"),
                                  bg="#34495e",
                                  fg="white",
                                  padx=15,
                                  pady=10)
        algo_frame.pack(side=tk.LEFT, padx=10)
        
        self.algo_var = tk.StringVar(value="minimax_ab")
        algorithms = [("Minimax", "minimax"), ("Minimax + Alpha-Beta", "minimax_ab")]
        for text, value in algorithms:
            radio = tk.Radiobutton(algo_frame, 
                                  text=text, 
                                  variable=self.algo_var, 
                                  value=value,
                                  font=("Arial", 11),
                                  bg="#34495e",
                                  fg="white",
                                  selectcolor="#2c3e50",
                                  activebackground="#34495e",
                                  activeforeground="#3498db")
            radio.pack(anchor="w", pady=2)
        
        # زر البدء
        button_frame = tk.Frame(menu_frame, bg="#34495e")
        button_frame.pack(pady=30)
        
        start_btn = tk.Button(button_frame, 
                             text="🚀 Start Game", 
                             font=("Arial", 16, "bold"),
                             command=self.start_game, 
                             bg="#2ecc71",
                             fg="white",
                             activebackground="#27ae60",
                             activeforeground="white",
                             padx=30,
                             pady=10,
                             relief="raised",
                             bd=3)
        start_btn.pack()
        
        # نص توضيحي أسفل الزر
        footer_label = tk.Label(menu_frame,
                              text="Designed with ❤️ | Connect 4 AI Visualization",
                              font=("Arial", 10),
                              bg="#34495e",
                              fg="#7f8c8d")
        footer_label.pack(pady=(20, 0))
    
    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()
    
    def start_game(self):
        self.depth = self.depth_var.get()
        self.algorithm = self.algo_var.get()
        self.game = Connect4Game()
        self.ai = Connect4AI(self.game, self.depth, self.algorithm == "minimax_ab")
        self.setup_game_board()
    
    def setup_game_board(self):
        self.clear_window()
        
        # إطار رئيسي للعبة
        main_frame = tk.Frame(self.root, bg="#2c3e50")
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # شريط المعلومات العلوي
        header_frame = tk.Frame(main_frame, bg="#34495e", relief="ridge", bd=2, padx=15, pady=10)
        header_frame.pack(fill=tk.X, pady=(0, 10))
        
        # إطار للإحصائيات في نفس الشكل
        stats_container = tk.Frame(header_frame, bg="#2c3e50", relief="solid", bd=1, padx=15, pady=8)
        stats_container.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        self.stats_label = tk.Label(stats_container, 
                                   text=f"🏆  Human: {self.human_wins} wins  |  AI: {self.ai_wins} wins  |  Draws: {self.draws}  🏆", 
                                   font=("Arial", 11, "bold"),
                                   bg="#2c3e50",
                                   fg="#f1c40f")
        self.stats_label.pack()
        
        settings_info = tk.Label(stats_container,
                               text=f"⚙️  Depth: {self.depth}  |  Algorithm: {self.algorithm}",
                               font=("Arial", 10),
                               bg="#2c3e50",
                               fg="#bdc3c7")
        settings_info.pack()
        
        # إطار معلومات الدور
        turn_frame = tk.Frame(header_frame, bg="#34495e", relief="solid", bd=1, padx=10, pady=8)
        turn_frame.pack(side=tk.RIGHT)
        
        self.turn_label = tk.Label(turn_frame, 
                                  text="Human's Turn (Red)", 
                                  font=("Arial", 12, "bold"),
                                  bg="#34495e",
                                  fg="#e74c3c")
        self.turn_label.pack()
        
        # أزرار الأعمدة
        button_frame = tk.Frame(main_frame, bg="#2c3e50")
        button_frame.pack(pady=(0, 10))
        
        self.buttons = []
        for col in range(self.game.cols):
            btn = tk.Button(button_frame, 
                           text="↓", 
                           font=("Arial", 14, "bold"),
                           command=lambda c=col: self.human_move(c), 
                           width=3, 
                           height=1, 
                           bg="#3498db",
                           fg="white",
                           activebackground="#2980b9",
                           activeforeground="white",
                           relief="raised",
                           bd=3)
            btn.grid(row=0, column=col, padx=3)
            self.buttons.append(btn)
        
        # لوحة اللعب
        board_container = tk.Frame(main_frame, bg="#2980b9", padx=10, pady=10, relief="sunken", bd=4)
        board_container.pack(pady=10)
        
        self.circles = []
        for row in range(self.game.rows):
            circle_row = []
            for col in range(self.game.cols):
                canvas = tk.Canvas(board_container, width=70, height=70, bg="#2980b9", 
                                 highlightthickness=0)
                canvas.grid(row=row, column=col, padx=3, pady=3)
                
                # إضافة تأثير ظل للدائرة
                canvas.create_oval(8, 8, 66, 66, fill="#1c4e80", outline="#1c4e80")
                
                # الدائرة الرئيسية
                circle = canvas.create_oval(5, 5, 65, 65, fill="white", 
                                          outline="#34495e", width=3)
                circle_row.append((canvas, circle))
            self.circles.append(circle_row)
        
        # أزرار التحكم - تصميم متسق
        control_frame = tk.Frame(main_frame, bg="#2c3e50", pady=15)
        control_frame.pack()
        
        # أزرار بنفس التصميم
        button_style = {
            "font": ("Arial", 12, "bold"),
            "fg": "white",
            "activeforeground": "white",
            "relief": "raised",
            "bd": 3,
            "padx": 20,
            "pady": 8
        }
        
        # زر القائمة الرئيسية
        menu_btn = tk.Button(control_frame, 
                            text="🏠 Main Menu", 
                            command=self.setup_menu, 
                            bg="#e67e22",
                            activebackground="#d35400",
                            **button_style)
        menu_btn.pack(side=tk.LEFT, padx=5)
        
        # زر عرض الشجرة الكبيرة
        self.show_tree_btn = tk.Button(control_frame, 
                                      text="🌳 Show Large Tree", 
                                      command=self.show_tree_visual, 
                                      bg="#9b59b6",
                                      activebackground="#8e44ad",
                                      state=tk.DISABLED,
                                      **button_style)
        self.show_tree_btn.pack(side=tk.LEFT, padx=5)
        
        # زر عرض الإحصائيات النهائية
        self.show_stats_btn = tk.Button(control_frame,
                                      text="📊 Show Final Stats",
                                      command=self.show_game_complete_message,
                                      bg="#3498db",
                                      activebackground="#2980b9",
                                      state=tk.DISABLED,
                                      **button_style )
        self.show_stats_btn.pack(side=tk.LEFT, padx=10)
        
        # وحدة التحكم للنص
        console_frame = tk.LabelFrame(main_frame, 
                                     text="📝 Game Console", 
                                     font=("Arial", 11, "bold"),
                                     bg="#2c3e50",
                                     fg="white",
                                     padx=10,
                                     pady=10)
        console_frame.pack(fill=tk.BOTH, expand=True, pady=(10, 0))
        
        text_frame = tk.Frame(console_frame, bg="#34495e")
        text_frame.pack(fill=tk.BOTH, expand=True)
        
        self.console_text = tk.Text(text_frame, 
                                   height=8, 
                                   bg="#1a252f", 
                                   fg="white", 
                                   font=("Consolas", 10),
                                   relief="flat",
                                   wrap="word")
        scrollbar = tk.Scrollbar(text_frame, 
                                command=self.console_text.yview,
                                bg="#34495e",
                                troughcolor="#2c3e50")
        self.console_text.config(yscrollcommand=scrollbar.set)
        self.console_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.redirect_console()
        self.update_board()
        
        # طباعة رسالة بداية اللعبة
        print(f"\n{'='*60}")
        print("🎮 STARTING CONNECT 4 GAME")
        print(f"{'='*60}")
        print(f"⚙️  Settings: Depth={self.depth}, Algorithm={self.algorithm}")
        print("📌 Game continues until board is completely filled.")
        print("🌳 Click 'Show Large Tree' at the end to see BIG graphical decision tree.")
        print(f"{'='*60}\n")
    
    def redirect_console(self):
        import sys
        class ConsoleRedirector:
            def __init__(self, text_widget):
                self.text_widget = text_widget
            def write(self, message):
                self.text_widget.insert(tk.END, message)
                self.text_widget.see(tk.END)
                self.text_widget.update_idletasks()
            def flush(self):
                pass
        sys.stdout = ConsoleRedirector(self.console_text)
    
    def update_board(self):
        for row in range(self.game.rows):
            for col in range(self.game.cols):
                canvas, circle = self.circles[row][col]
                piece = self.game.board[row][col]
                if piece == 1:
                    canvas.itemconfig(circle, fill="#e74c3c")  # أحمر
                elif piece == 2:
                    canvas.itemconfig(circle, fill="#f1c40f")  # أصفر ذهبي
                else:
                    canvas.itemconfig(circle, fill="#ecf0f1")  # أبيض فاتح
                    
        if not self.game.game_over:
            if self.game.turn == 0:
                self.turn_label.config(text="Human's Turn (Red)", fg="#e74c3c")
                for btn in self.buttons:
                    btn.config(state=tk.NORMAL, bg="#3498db")
            elif self.game.turn == 1:
                self.turn_label.config(text="AI's Turn (Yellow)", fg="#f1c40f")
                for btn in self.buttons:
                    btn.config(state=tk.DISABLED, bg="#7f8c8d")
        else:
            # Game over state
            self.turn_label.config(text="🎮 Game Complete!", fg="#2ecc71")
            for btn in self.buttons:
                btn.config(state=tk.DISABLED, bg="#95a5a6")
    
    def place_piece(self, col, piece):
        if not self.game.is_valid_location(col):
            return False
            
        actual_row = self.game.drop_piece(col, piece)
        if actual_row == -1:
            return False
            
        self.total_moves += 1
        self.update_board()
        return True
    
    def check_win_and_continue(self, piece):
        """التحقق من الفوز لكن استمرار اللعبة"""
        if self.game.winning_move(piece):
            if piece == 1:  # Human
                self.human_wins += 1
                print(f"\n{'='*40}")
                print(f"🎉 HUMAN SCORES A WIN!")
                print(f"Total human wins: {self.human_wins}")
                print(f"{'='*40}")
                self.update_stats_label()
            else:  # AI
                self.ai_wins += 1
                print(f"\n{'='*40}")
                print(f"🤖 AI SCORES A WIN!")
                print(f"Total AI wins: {self.ai_wins}")
                print(f"{'='*40}")
                self.update_stats_label()
            return True
        return False
    
    def update_stats_label(self):
        """تحديث تسمية الإحصائيات"""
        self.stats_label.config(text=f"🏆  Human: {self.human_wins} wins  |  AI: {self.ai_wins} wins  |  Draws: {self.draws}  🏆")
    
    def human_move(self, col):
        if self.game.game_over:
            return
            
        if self.place_piece(col, 1):
            # التحقق من الفوز
            self.check_win_and_continue(1)
            
            # التحقق إذا كانت اللوحة ممتلئة
            if self.game.is_board_full():
                self.end_complete_game()
                return
            
            # استمر في اللعب
            self.game.turn = 1
            self.update_board()
            
            # إذا لم تنته اللعبة، دع الـ AI يلعب
            if not self.game.game_over:
                self.root.after(500, self.ai_move)
    
    def ai_move(self):
        if self.game.game_over:
            return
            
        # طباعة معلومات حول حركة الـ AI
        print(f"\n{'='*60}")
        print(f"🤖 AI MOVE #{self.total_moves//2 + 1}")
        print(f"{'='*60}")
        
        col = self.ai.get_best_move()
        
        if col != -1:  # إذا كانت هناك حركة متاحة
            print(f"✅ AI chose column: {col + 1}")
            print(f"📊 Tree Statistics:")
            print(f"   Total nodes in tree: {self.ai.total_nodes}")
            print(f"   Execution time: {self.ai.execution_time:.3f}s")
            
            if self.place_piece(col, 2):
                # التحقق من الفوز
                self.check_win_and_continue(2)
                
                # التحقق إذا كانت اللوحة ممتلئة
                if self.game.is_board_full():
                    self.end_complete_game()
                    return
                
                # استمر في اللعب
                self.game.turn = 0
                self.update_board()
        else:
            print("❌ No valid moves for AI")
    
    def end_complete_game(self):
        """نهاية اللعبة بعد امتلاء اللوحة تماماً"""
        self.game.game_over = True
        
        # تفعيل الأزرار
        self.show_tree_btn.config(state=tk.NORMAL, bg="#9b59b6")
        self.show_stats_btn.config(state=tk.NORMAL, bg="#3498db")
        
        # حساب التعادلات
        total_wins = self.human_wins + self.ai_wins
        total_rounds = self.total_moves // 2
        self.draws = total_rounds - total_wins
        
        # تحديث الإحصائيات
        self.update_stats_label()
        self.update_board()
        
        # طباعة النتائج النهائية
        print(f"\n{'='*60}")
        print("🎮 GAME COMPLETE - BOARD IS FULL!")
        print(f"{'='*60}")
        print(f"\n📊 FINAL GAME STATISTICS:")
        print(f"   Total moves: {self.total_moves}")
        print(f"   Total rounds: {total_rounds}")
        print(f"   Human wins: {self.human_wins}")
        print(f"   AI wins: {self.ai_wins}")
        print(f"   Draws: {self.draws}")
        
        # تحديد الفائز النهائي
        if self.human_wins > self.ai_wins:
            print(f"\n🏆 OVERALL WINNER: HUMAN! 🎉")
        elif self.ai_wins > self.human_wins:
            print(f"\n🏆 OVERALL WINNER: AI! 🤖")
        else:
            print(f"\n🏆 OVERALL RESULT: DRAW! 🤝")
        print(f"\n{'='*60}")
        
        # تغيير تسمية الدور
        self.turn_label.config(text="🎮 Game Complete!", fg="#2ecc71")
        
        # عرض رسالة الإنهاء بنفس تصميم الإحصائيات النهائية
        self.show_game_complete_message()
    
    def show_game_complete_message(self):
        """عرض رسالة اكتمال اللعبة بتصميم أنيق"""
        # إنشاء نافذة مخصصة للإحصائيات النهائية
        stats_window = tk.Toplevel(self.root)
        stats_window.title("🎮 Game Complete!")
        stats_window.geometry("500x650")
        stats_window.configure(bg="#2c3e50")
        stats_window.resizable(False, False)
        
        # إطار رئيسي
        main_frame = tk.Frame(stats_window, bg="#34495e", relief="ridge", bd=3, padx=20, pady=20)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # العنوان
        title_label = tk.Label(main_frame,
                              text="🎮 GAME COMPLETE!",
                              font=("Arial", 22, "bold"),
                              bg="#34495e",
                              fg="#2ecc71")
        title_label.pack(pady=(0, 20))
        
        # إطار الإحصائيات
        stats_frame = tk.Frame(main_frame, bg="#2c3e50", relief="solid", bd=2, padx=15, pady=15)
        stats_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # العنوان الفرعي
        sub_title = tk.Label(stats_frame,
                           text="📊 FINAL GAME STATISTICS",
                           font=("Arial", 14, "bold"),
                           bg="#2c3e50",
                           fg="white")
        sub_title.pack(pady=(0, 15))
        
        # إحصائيات اللعبة
        stats_data = [
            ("🎯 Total Moves:", f"{self.total_moves}"),
            ("🔄 Total Rounds:", f"{self.total_moves // 2}"),
            ("", ""),  # فاصل
            ("👤 Human Wins:", f"{self.human_wins}"),
            ("🤖 AI Wins:", f"{self.ai_wins}"),
            ("🤝 Draws:", f"{self.draws}"),
            ("", ""),  # فاصل
            ("⚙️ Search Depth:", f"{self.depth}"),
            ("🧠 Algorithm:", "Minimax with Alpha-Beta" if self.algorithm == "minimax_ab" else "Minimax")
        ]
        
        for label_text, value_text in stats_data:
            if label_text == "":  # فاصل
                tk.Frame(stats_frame, height=10, bg="#2c3e50").pack()
                continue
                
            stat_row = tk.Frame(stats_frame, bg="#2c3e50")
            stat_row.pack(fill=tk.X, pady=3)
            
            label = tk.Label(stat_row,
                           text=label_text,
                           font=("Arial", 11),
                           bg="#2c3e50",
                           fg="#bdc3c7",
                           anchor="w")
            label.pack(side=tk.LEFT)
            
            value = tk.Label(stat_row,
                           text=value_text,
                           font=("Arial", 11, "bold"),
                           bg="#2c3e50",
                           fg="#f1c40f",
                           anchor="e")
            value.pack(side=tk.RIGHT)
        
        # الفائز النهائي
        winner_frame = tk.Frame(main_frame, bg="#34495e", pady=15)
        winner_frame.pack(fill=tk.X)
        
        if self.human_wins > self.ai_wins:
            winner_text = "🏆 HUMAN WINS THE GAME! 🎉"
            winner_color = "#e74c3c"
        elif self.ai_wins > self.human_wins:
            winner_text = "🏆 AI WINS THE GAME! 🤖"
            winner_color = "#f1c40f"
        else:
            winner_text = "🏆 IT'S A DRAW! 🤝"
            winner_color = "#2ecc71"
        
        winner_label = tk.Label(winner_frame,
                              text=winner_text,
                              font=("Arial", 16, "bold"),
                              bg="#34495e",
                              fg=winner_color)
        winner_label.pack()
        
        # أزرار الإجراءات
        button_frame = tk.Frame(main_frame, bg="#34495e", pady=10)
        button_frame.pack()
        
        tk.Button(button_frame,
                 text="🌳 View Decision Tree",
                 font=("Arial", 11, "bold"),
                 command=lambda: [stats_window.destroy(), self.show_tree_visual()],
                 bg="#9b59b6",
                 fg="white",
                 activebackground="#8e44ad",
                 activeforeground="white",
                 padx=15,
                 pady=5,
                 relief="raised",
                 bd=2).pack(side=tk.LEFT, padx=5)
        
        tk.Button(button_frame,
                 text="👋 Close",
                 font=("Arial", 11),
                 command=stats_window.destroy,
                 bg="#95a5a6",
                 fg="white",
                 activebackground="#7f8c8d",
                 activeforeground="white",
                 padx=15,
                 pady=5,
                 relief="raised",
                 bd=2).pack(side=tk.LEFT, padx=5)
    
    def show_tree_visual(self):
        """عرض الشجرة كرسمة بيانية كبيرة"""
        if self.ai and self.ai.last_move_tree:
            try:
                # إنشاء مصور الشجرة الكبيرة
                algorithm_name = "Minimax with Alpha-Beta" if self.algorithm == "minimax_ab" else "Minimax"
                TreeVisualizer(self.ai.last_move_tree, self.depth, algorithm_name)
                print("\n✅ Large tree visualization window opened!")
                print("   Use scrollbars to navigate and zoom buttons to adjust size.")
            except Exception as e:
                print(f"\n❌ Error creating tree visualization: {e}")
                messagebox.showerror("Error", f"Could not create tree visualization:\n{str(e)}")
        else:
            messagebox.showwarning("No Tree Data", "No tree data available to visualize.")
    
    def show_final_statistics(self):
        """إظهار الإحصائيات النهائية بنفس تصميم رسالة اكتمال اللعبة"""
        # إنشاء نافذة مخصصة للإحصائيات النهائية
        stats_window = tk.Toplevel(self.root)
        stats_window.title("📊 Final Game Statistics")
        stats_window.geometry("500x")
        stats_window.configure(bg="#2c3e50")
        stats_window.resizable(False, False)
        
        # إطار رئيسي
        main_frame = tk.Frame(stats_window, bg="#34495e", relief="ridge", bd=3, padx=20, pady=20)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        # العنوان
        title_label = tk.Label(main_frame,
                              text="📊 FINAL STATISTICS",
                              font=("Arial", 22, "bold"),
                              bg="#34495e",
                              fg="#3498db")
        title_label.pack(pady=(0, 20))
        
        # إطار الإحصائيات
        stats_frame = tk.Frame(main_frame, bg="#2c3e50", relief="solid", bd=2, padx=15, pady=15)
        stats_frame.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # العنوان الفرعي
        sub_title = tk.Label(stats_frame,
                           text="🎮 GAME SUMMARY",
                           font=("Arial", 14, "bold"),
                           bg="#2c3e50",
                           fg="white")
        sub_title.pack(pady=(0, 15))
        
        # إحصائيات اللعبة
        stats_data = [
            ("🎯 Total Moves:", f"{self.total_moves}"),
            ("🔄 Total Rounds:", f"{self.total_moves // 2}"),
            ("", ""),  # فاصل
            ("👤 Human Wins:", f"{self.human_wins}"),
            ("🤖 AI Wins:", f"{self.ai_wins}"),
            ("🤝 Draws:", f"{self.draws}"),
            ("", ""),  # فاصل
            ("⚙️ Search Depth:", f"{self.depth}"),
            ("🧠 Algorithm:", "Minimax with Alpha-Beta" if self.algorithm == "minimax_ab" else "Minimax")
        ]
        
        for label_text, value_text in stats_data:
            if label_text == "":  # فاصل
                tk.Frame(stats_frame, height=10, bg="#2c3e50").pack()
                continue
                
            stat_row = tk.Frame(stats_frame, bg="#2c3e50")
            stat_row.pack(fill=tk.X, pady=3)
            
            label = tk.Label(stat_row,
                           text=label_text,
                           font=("Arial", 11),
                           bg="#2c3e50",
                           fg="#bdc3c7",
                           anchor="w")
            label.pack(side=tk.LEFT)
            
            value = tk.Label(stat_row,
                           text=value_text,
                           font=("Arial", 11, "bold"),
                           bg="#2c3e50",
                           fg="#f1c40f",
                           anchor="e")
            value.pack(side=tk.RIGHT)
        
        # النتيجة النهائية
        result_frame = tk.Frame(main_frame, bg="#34495e", pady=15)
        result_frame.pack(fill=tk.X)
        
        if self.human_wins > self.ai_wins:
            result_text = "🎉 FINAL WINNER: HUMAN!"
            result_color = "#e74c3c"
        elif self.ai_wins > self.human_wins:
            result_text = "🤖 FINAL WINNER: AI!"
            result_color = "#f1c40f"
        else:
            result_text = "🤝 FINAL RESULT: DRAW!"
            result_color = "#2ecc71"
        
        result_label = tk.Label(result_frame,
                              text=result_text,
                              font=("Arial", 16, "bold"),
                              bg="#34495e",
                              fg=result_color)
        result_label.pack()
        
        # زر الإغلاق
        close_frame = tk.Frame(main_frame, bg="#34495e", pady=10)
        close_frame.pack()
        
        tk.Button(close_frame,
                 text="👋 Close",
                 font=("Arial", 11),
                 command=stats_window.destroy,
                 bg="#95a5a6",
                 fg="white",
                 activebackground="#7f8c8d",
                 activeforeground="white",
                 padx=20,
                 pady=5,
                 relief="raised",
                 bd=2).pack()
    
    def run(self):
        self.root.mainloop()

def main():
    app = Connect4GUI()
    app.run()

if __name__ == "__main__":
    main()